## シミュレーションを流し、学習するファイル

In [1]:
#path
pdb_filepath = "./alanine-dipeptide-nowater.pdb"
ff_filepath  = "./output.xml" #startのffのパス
out_dir = "./outdir/"
isdir(out_dir) || mkdir(out_dir)

#setting
username = "ysako" #自分のusername
n_jobs = 5 #並列して流すシミュレーションの数
nsteps = 5000000 #シミュレーションのステップ数
epoch_n = 0 #epoch_nは学習を繰り返した数

5000000

In [2]:
# 一つのシミュレーションを流すジョブスクリプト
# ff_filepath　　　  :xml formatの力場ファイルのパス
# epoch_dir         :アウトプットするディレクトリのパス
# i                 :流すシミュレーションの通し番号

function run_job(ff_filepath, epoch_n, epoch_dir, i)
    sbatch_file = "temp_epoch$(epoch_n)_$(i).sh"
    traj_filepath = joinpath(epoch_dir, "traj")
    
    open(sbatch_file, "w") do of
        println(of, "#!/bin/bash")
        println(of, "#SBATCH -p all")
        println(of, "#SBATCH -J sim$(i) # job name")
        println(of, "#SBATCH -n 1  # num of total mpi processes")
        println(of, "#SBATCH -c 1  # num of threads per mpi processes")
        println(of, "#SBATCH -o $(epoch_dir)/run$(i).log")
        println(of, "#SBATCH -w n4")
        println(of, "python sim.py $(pdb_filepath) $(ff_filepath) $(traj_filepath)$(i) $(nsteps)")
    end
    run(`sbatch $(sbatch_file)`)
    sleep(5)
    rm(sbatch_file)
end

run_job (generic function with 1 method)

In [3]:
#epoch_n
function run_jobs(epoch_n)
    epoch_dir = joinpath(out_dir, "epoch_$(epoch_n)") #epoch_nで出力するディレクトリ
    isdir(epoch_dir) || mkdir(epoch_dir)

    completed = false #全てのシミュレーションを終えたらtrueに

    #njobsの数だけシミュレーションを並列に流す
    for i in 1:n_jobs
        run_job(ff_filepath, epoch_n, epoch_dir, i)
    end

    #シミュレーションが全て終わるまで待機
    while(!completed)
        n_completed = n_jobs - length(readlines(`squeue -u $(username)`)) + 1 #終了したシミュレーションの数
        #シミュレーションが終わっていないとき
        if(n_completed < n_jobs)
            println("--- $(n_completed) of $(n_jobs) jobs completed")
            sleep(120) #2分待機
        #全てのシミュレーションが終わったとき
        else
            completed = true
        end
    end
    println("--- all jobs completed")
end

run_jobs (generic function with 1 method)

In [ ]:
epoch_n = 1
run_jobs(epoch_n)

Submitted batch job 101
Submitted batch job 102
Submitted batch job 103
Submitted batch job 104
Submitted batch job 105
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs completed
--- -1 of 5 jobs

In [17]:
readlines(`squeue -u $(username)`)

6-element Vector{String}:
 "             JOBID PARTITION   " ⋯ 22 bytes ⋯ "   TIME  NODES NODELIST(REASON)"
 "                83       all     sim1    ysako  R       2:16      1 n4"
 "                84       all     sim2    ysako  R       2:11      1 n4"
 "                85       all     sim3    ysako  R       2:06      1 n4"
 "                86       all     sim4    ysako  R       2:01      1 n4"
 "                87       all     sim5    ysako  R       1:56      1 n4"